In [1]:
#-*- coding:utf-8 -*-
import numpy as np
import numpy.random as rd
import pandas as pd

seed = 0

In [2]:
#S: 状態空間, |S|=N1
N1 = 2

#Y: イベントシンボル集合 |Y|=N2
Y = [0, 1]
print(Y)
N2 = len(Y)

[0, 1]


In [3]:
#y: 長さTのセッション
T = 10
rd.seed(0)
y = rd.choice(list(Y), T)
print(y)

[0 1 1 0 1 1 1 1 1 1]


### Baum-Welch

In [4]:
#E-Step: (α, β, τ, τ'の推定)
#初期化
gamma = np.array([1/N1 for _ in range(N1)]) #等確率とする
A = np.ones((N1, N1)) / N1
B = np.ones((N1, N2)) / N2
alpha = np.zeros((T, N1))
beta = np.zeros((T, N1))
tau = np.zeros((T, N1, N1))
tau_ = np.zeros((T, N1))

In [5]:
A

array([[ 0.5,  0.5],
       [ 0.5,  0.5]])

In [6]:
B

array([[ 0.5,  0.5],
       [ 0.5,  0.5]])

In [7]:
gamma

array([ 0.5,  0.5])

In [8]:
#αについて(forward型推定)
#Step1: 初期化
alpha[0] = gamma * B[:, y[0]]

#Step2: 再帰的計算
for t in range(T-1):
    for j in range(N1):
        alpha[t+1, j] = np.sum(alpha[t] * A[:, j]) * B[j, y[t+1]]

In [9]:
#βについて(backward推定)
#Step1: 初期化
beta[-1] = 1

#Step2: 再帰的計算
for t in range(T-1)[::-1]:
    for i in range(N1):
        beta[t, i] = np.sum(A[i] * B[:, y[t+1]] * beta[t+1])

In [10]:
alpha

array([[ 0.25      ,  0.25      ],
       [ 0.125     ,  0.125     ],
       [ 0.0625    ,  0.0625    ],
       [ 0.03125   ,  0.03125   ],
       [ 0.015625  ,  0.015625  ],
       [ 0.0078125 ,  0.0078125 ],
       [ 0.00390625,  0.00390625],
       [ 0.00195312,  0.00195312],
       [ 0.00097656,  0.00097656],
       [ 0.00048828,  0.00048828]])

In [11]:
beta

array([[ 0.00195312,  0.00195312],
       [ 0.00390625,  0.00390625],
       [ 0.0078125 ,  0.0078125 ],
       [ 0.015625  ,  0.015625  ],
       [ 0.03125   ,  0.03125   ],
       [ 0.0625    ,  0.0625    ],
       [ 0.125     ,  0.125     ],
       [ 0.25      ,  0.25      ],
       [ 0.5       ,  0.5       ],
       [ 1.        ,  1.        ]])

In [12]:
y

array([0, 1, 1, 0, 1, 1, 1, 1, 1, 1])

In [13]:
y = [0 for i in range(T)]
y

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [14]:
#τについて
for t in range(T-1):
    #分母(denominator)を計算する
    d = np.sum([np.sum(alpha[t, i] * A[i, :] * B[:, y[t+1]] * beta[t+1, :]) for i in range(N1)])
    print(d)
    
    for i in range(N1):
        for j in range(N1):
            tau[t, i, j] = alpha[t, i] * A[i, j] * B[j, y[t+1]] * beta[t+1, j] / d

0.0009765625
0.0009765625
0.0009765625
0.0009765625
0.0009765625
0.0009765625
0.0009765625
0.0009765625
0.0009765625


In [15]:
t = 0
alpha[t, 0] * A[0,0] * B[0, y[t+1]] * beta[t+1,0] + alpha[t, 1] * A[0,1] * B[1, y[t+1]] * beta[t+1,1] +\
alpha[t, 1] * A[1,0] * B[1, y[t+1]] * beta[t+1,1] + alpha[t, 1] * A[1,1] * B[1, y[t+1]] * beta[t+1,1] 

0.0009765625

In [16]:
t = 8
alpha[t, 0] * A[0,0] * B[0, y[t+1]] * beta[t+1,0] + alpha[t, 1] * A[0,1] * B[1, y[t+1]] * beta[t+1,1] +\
alpha[t, 1] * A[1,0] * B[1, y[t+1]] * beta[t+1,1] + alpha[t, 1] * A[1,1] * B[1, y[t+1]] * beta[t+1,1]

0.0009765625

In [17]:
tau

array([[[ 0.25,  0.25],
        [ 0.25,  0.25]],

       [[ 0.25,  0.25],
        [ 0.25,  0.25]],

       [[ 0.25,  0.25],
        [ 0.25,  0.25]],

       [[ 0.25,  0.25],
        [ 0.25,  0.25]],

       [[ 0.25,  0.25],
        [ 0.25,  0.25]],

       [[ 0.25,  0.25],
        [ 0.25,  0.25]],

       [[ 0.25,  0.25],
        [ 0.25,  0.25]],

       [[ 0.25,  0.25],
        [ 0.25,  0.25]],

       [[ 0.25,  0.25],
        [ 0.25,  0.25]],

       [[ 0.  ,  0.  ],
        [ 0.  ,  0.  ]]])

In [18]:
#τ'について
for t in range(T):
    for i in range(N1):
        tau_[t, i] = np.sum(tau[t][i, :])

In [19]:
tau_

array([[ 0.5,  0.5],
       [ 0.5,  0.5],
       [ 0.5,  0.5],
       [ 0.5,  0.5],
       [ 0.5,  0.5],
       [ 0.5,  0.5],
       [ 0.5,  0.5],
       [ 0.5,  0.5],
       [ 0.5,  0.5],
       [ 0. ,  0. ]])

In [20]:
def Baum_Welch(y, T, N1, N2):
    '''
    Baum_WelchにおけるE-Step
    '''
    #初期化
    gamma = np.array([1/N1 for _ in range(N1)]) #等確率とする
    A = np.ones((N1, N1)) / N1
    B = np.ones((N1, N2)) / N2
    alpha = np.zeros((T, N1))
    beta = np.zeros((T, N1))
    tau = np.zeros((T, N1, N1))
    tau_ = np.zeros((T, N1))
    
    #αについて(forward型推定)
    #Step1: 初期化
    alpha[0] = gamma * B[:, y[0]]

    #Step2: 再帰的計算
    for t in range(T-1):
        for j in range(N1):
            alpha[t+1, j] = np.sum(alpha[t] * A[:, j]) * B[j, y[t+1]]
    
    
    #βについて(backward推定)
    #Step1: 初期化
    beta[-1] = 1

    #Step2: 再帰的計算
    for t in range(T-1)[::-1]:
        for i in range(N1):
            beta[t, i] = np.sum(A[i] * B[:, y[t+1]] * beta[t+1])
    
    
    #τについて
    for t in range(T-1):
        #分母(denominator)を計算する
        d = np.sum([np.sum(alpha[t, i] * A[i, :] * B[:, y[t+1]] * beta[t+1, :]) for i in range(N1)])

        for i in range(N1):
            for j in range(N1):
                tau[t, i, j] = alpha[t, i] * A[i, j] * B[j, y[t+1]] * beta[t+1, j] / d
    
    
    #τ'について
    for t in range(T):
        for i in range(N1):
            tau_[t, i] = np.sum(tau[t][i, :])
            
    return alpha, beta, tau, tau_

In [21]:
#S: 状態空間, |S|=N1
N1 = 2

#Y: イベントシンボル集合 |Y|=N2
Y = [0, 1]
print(Y)
N2 = len(Y)

#y: 長さTのセッション
T = 10
rd.seed(0)
y = rd.choice(list(Y), T)
print(y)

[0, 1]
[0 1 1 0 1 1 1 1 1 1]


In [22]:
Baum_Welch(y, T, N1, N2)

(array([[ 0.25      ,  0.25      ],
        [ 0.125     ,  0.125     ],
        [ 0.0625    ,  0.0625    ],
        [ 0.03125   ,  0.03125   ],
        [ 0.015625  ,  0.015625  ],
        [ 0.0078125 ,  0.0078125 ],
        [ 0.00390625,  0.00390625],
        [ 0.00195312,  0.00195312],
        [ 0.00097656,  0.00097656],
        [ 0.00048828,  0.00048828]]), array([[ 0.00195312,  0.00195312],
        [ 0.00390625,  0.00390625],
        [ 0.0078125 ,  0.0078125 ],
        [ 0.015625  ,  0.015625  ],
        [ 0.03125   ,  0.03125   ],
        [ 0.0625    ,  0.0625    ],
        [ 0.125     ,  0.125     ],
        [ 0.25      ,  0.25      ],
        [ 0.5       ,  0.5       ],
        [ 1.        ,  1.        ]]), array([[[ 0.25,  0.25],
         [ 0.25,  0.25]],
 
        [[ 0.25,  0.25],
         [ 0.25,  0.25]],
 
        [[ 0.25,  0.25],
         [ 0.25,  0.25]],
 
        [[ 0.25,  0.25],
         [ 0.25,  0.25]],
 
        [[ 0.25,  0.25],
         [ 0.25,  0.25]],
 
        [[ 0.